<a href="https://colab.research.google.com/github/kaavyaastalin/AgeGenderEmotion/blob/master/ThesisFaceTesting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/kaushikjadhav01/Deep-Surveillance-Monitor-Facial-Emotion-Age-Gender-Recognition-System.git

Cloning into 'Deep-Surveillance-Monitor-Facial-Emotion-Age-Gender-Recognition-System'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 47 (delta 10), reused 26 (delta 0), pack-reused 0
Unpacking objects: 100% (47/47), done.


In [ ]:
import os
RESEARCH_DIR = "/content/Deep-Surveillance-Monitor-Facial-Emotion-Age-Gender-Recognition-System"
os.chdir(RESEARCH_DIR)
os.getcwd()

'/content/Deep-Surveillance-Monitor-Facial-Emotion-Age-Gender-Recognition-System'

In [ ]:
!pip install -r requirements.txt

In [ ]:
from pathlib import Path
import cv2
import dlib
import sys
import numpy as np
import argparse
from contextlib import contextmanager
from wide_resnet import WideResNet
from keras.utils.data_utils import get_file
from keras.models import load_model
from keras.preprocessing.image import img_to_array

classifier = load_model('./model/emotion_little_vgg_2.h5')
pretrained_model = "https://github.com/yu4u/age-gender-estimation/releases/download/v0.5/weights.28-3.73.hdf5"

In [109]:
from os import listdir
from os.path import isfile, join
import os
import csv
import cv2
import json
from google.colab.patches import cv2_imshow
# Define Image Path Here
image_path = "./images/"

modhash = 'fbe63257a054c1c5466cfd7bf14646d6'
emotion_classes = {0: 'Angry', 1: 'Fear', 2: 'Happy', 3: 'Neutral', 4: 'Sad', 5: 'Surprise'}

def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=0.8, thickness=1):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)
    

# Define our model parameters
depth = 16
k = 8
weight_file = None
margin = 0.4
image_dir = None

# Get our weight file 
if not weight_file:
    weight_file = get_file("weights.28-3.73.hdf5", pretrained_model, cache_subdir="pretrained_models",
                           file_hash=modhash, cache_dir=Path(sys.argv[0]).resolve().parent)
# load model and weights
img_size = 64
model = WideResNet(img_size, depth=depth, k=k)()
model.load_weights(weight_file)

detector = dlib.get_frontal_face_detector()

image_names = [f for f in listdir(image_path) if isfile(join(image_path, f))]

for image_name in image_names:
    frame = cv2.imread("./images/" + image_name)
    preprocessed_faces_emo = []           
 
    input_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_h, img_w, _ = np.shape(input_img)
    detected = detector(frame, 1)
    faces = np.empty((len(detected), img_size, img_size, 3))
    
    preprocessed_faces_emo = []
    if len(detected) > 0:
        for i, d in enumerate(detected):
            x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
            xw1 = max(int(x1 - margin * w), 0)
            yw1 = max(int(y1 - margin * h), 0)
            xw2 = min(int(x2 + margin * w), img_w - 1)
            yw2 = min(int(y2 + margin * h), img_h - 1)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            # cv2.rectangle(img, (xw1, yw1), (xw2, yw2), (255, 0, 0), 2)
            faces[i, :, :, :] = cv2.resize(frame[yw1:yw2 + 1, xw1:xw2 + 1, :], (img_size, img_size))
            face =  frame[yw1:yw2 + 1, xw1:xw2 + 1, :]
            face_gray_emo = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            face_gray_emo = cv2.resize(face_gray_emo, (48, 48), interpolation = cv2.INTER_AREA)
            face_gray_emo = face_gray_emo.astype("float") / 255.0
            face_gray_emo = img_to_array(face_gray_emo)
            face_gray_emo = np.expand_dims(face_gray_emo, axis=0)
            preprocessed_faces_emo.append(face_gray_emo)

        # make a prediction for Age and Gender
        results = model.predict(np.array(faces))
        predicted_genders = results[0]
        ages = np.arange(0, 101).reshape(101, 1)
        predicted_ages = results[1].dot(ages).flatten()
        #print(predicted_ages)

        # make a prediction for Emotion 
        emo_labels = []
        for i, d in enumerate(detected):
            preds = classifier.predict(preprocessed_faces_emo[i])[0]
            emo_labels.append(emotion_classes[preds.argmax()])
            #print(emo_labels)
        
        # draw results
        labels = []
        with open('HumanDetails.csv', 'a', newline='', encoding='utf-8') as out_file:
          for i, d in enumerate(detected):
            #labels = []

            
            label = "{}, {}, {}".format(int(predicted_ages[i]),
                                        "F" if predicted_genders[i][0] > 0.4 else "M", emo_labels[i])
            labels.append(label)
            draw_label(frame, (d.left(), d.top()), label)
            
            #resultDF = pd.DataFrame(labels)
          writer = csv.writer(out_file, delimiter=' ' )
            #writer.writerow(('age', 'gender','emotion'))
          writer.writerows(labels)
           #resultDF.to_csv('NewFilue.csv')
            #json.dump(labels, outfile)

    #cv2.imshow("Emotion Detector", frame)
    filename = "p/"+image_name
    cv2.imwrite(filename,frame)

    cv2.waitKey(0)

cv2.destroyAllWindows()

In [80]:
labels[0]\t  quotechar='|', quoting=csv.QUOTE_MINIMAL

'37, M, Happy'

In [ ]:
with open('data1.txt', 'r') as in_file:
    stripped = (line.strip() for line in in_file)
    lines = (line.split(",") for line in stripped if line)
    with open('log.csv', 'w', newline='') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(('age', 'gender','emotion'))
        writer.writerows(lines)

FileNotFoundError: ignored

In [110]:
import pandas as pd
sd = pd.read_csv('aaaa.csv', header=None)
sd

,0,1,2
0,5 9,""" "" M",""" "" N e u t r a l"
1,4 7,""" "" M",""" "" S a d"
2,3 7,""" "" F",""" "" H a p p y"
3,5 5,""" "" M",""" "" H a p p y"
4,6 2,""" "" M",""" "" H a p p y"
5,3 6,""" "" F",""" "" N e u t r a l"
6,4 1,""" "" F",""" "" H a p p y"
7,4 1,""" "" M",""" "" H a p p y"
8,3 7,""" "" M",""" "" H a p p y"


In [ ]:
from os import listdir
from os.path import isfile, join
import os
import cv2


modhash = 'fbe63257a054c1c5466cfd7bf14646d6'
emotion_classes = {0: 'Angry', 1: 'Fear', 2: 'Happy', 3: 'Neutral', 4: 'Sad', 5: 'Surprise'}

def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=0.8, thickness=1):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness, lineType=cv2.LINE_AA)
    

# Define our model parameters
depth = 16
k = 8
weight_file = None
margin = 0.4
image_dir = None

# Get our weight file 
if not weight_file:
    weight_file = get_file("weights.28-3.73.hdf5", pretrained_model, cache_subdir="pretrained_models",
                           file_hash=modhash, cache_dir=Path(sys.argv[0]).resolve().parent)
# load model and weights
img_size = 64
model = WideResNet(img_size, depth=depth, k=k)()
model.load_weights(weight_file)

detector = dlib.get_frontal_face_detector()

# Initialize Webcam
cap = cv2.VideoCapture('/content/Deep-Surveillance-Monitor-Facial-Emotion-Age-Gender-Recognition-System/videos/PP84_T1_H.mp4')
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output_video.mp4', fourcc, 10.0, (int(cap.get(3)), int(cap.get(4))))


while True:
    ret, frame = cap.read()
    

    preprocessed_faces_emo = []           
 
    input_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img_h, img_w, _ = np.shape(input_img)
    detected = detector(frame, 1)
    faces = np.empty((len(detected), img_size, img_size, 3))
    
    preprocessed_faces_emo = []
    if len(detected) > 0:
        for i, d in enumerate(detected):
            x1, y1, x2, y2, w, h = d.left(), d.top(), d.right() + 1, d.bottom() + 1, d.width(), d.height()
            xw1 = max(int(x1 - margin * w), 0)
            yw1 = max(int(y1 - margin * h), 0)
            xw2 = min(int(x2 + margin * w), img_w - 1)
            yw2 = min(int(y2 + margin * h), img_h - 1)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            # cv2.rectangle(img, (xw1, yw1), (xw2, yw2), (255, 0, 0), 2)
            faces[i, :, :, :] = cv2.resize(frame[yw1:yw2 + 1, xw1:xw2 + 1, :], (img_size, img_size))
            face =  frame[yw1:yw2 + 1, xw1:xw2 + 1, :]
            face_gray_emo = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            face_gray_emo = cv2.resize(face_gray_emo, (48, 48), interpolation = cv2.INTER_AREA)
            face_gray_emo = face_gray_emo.astype("float") / 255.0
            face_gray_emo = img_to_array(face_gray_emo)
            face_gray_emo = np.expand_dims(face_gray_emo, axis=0)
            preprocessed_faces_emo.append(face_gray_emo)

        # make a prediction for Age and Gender
        results = model.predict(np.array(faces))
        predicted_genders = results[0]
        ages = np.arange(0, 101).reshape(101, 1)
        predicted_ages = results[1].dot(ages).flatten()

        # make a prediction for Emotion 
        emo_labels = []
        for i, d in enumerate(detected):
            preds = classifier.predict(preprocessed_faces_emo[i])[0]
            emo_labels.append(emotion_classes[preds.argmax()])
        
        # draw results
        for i, d in enumerate(detected):
            label = "{}, {}, {}".format(int(predicted_ages[i]),
                                        "F" if predicted_genders[i][0] > 0.4 else "M", emo_labels[i])
            draw_label(frame, (d.left(), d.top()), label)

    out.write(frame)
    #cv2.imshow("Emotion Detector", frame)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break

cap.release()
out.release()
cv2.destroyAllWindows()

error: ignored